<a href="https://colab.research.google.com/github/WangMuying/E-F-Client-Info-Matching-Project/blob/main/pipeline_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import torch
from torch.utils.data import DataLoader, Dataset

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [18]:
def read_salesforce(file_path, encodings=['latin1', 'iso-8859-1', 'cp1252']):
    # Try reading the CSV with a different encoding
    for encoding in encodings:
        try:
            df_all = pd.read_csv(file_path, encoding=encoding)
            print(f"Successfully read the CSV file with {encoding} encoding.")
            break
        except UnicodeDecodeError:
            print(f"Failed to read the CSV file with {encoding} encoding.")
    else:
        print("Unable to read the CSV file with the specified encodings.")
    return df_all


def extract_name_from_email(email):
  name_part = email.split('@')[0]
  name_parts = re.split(r'[._]', name_part)
  name = ' '.join(name_parts).title()
  return name

def identify_and_replace_email_leads(df, salesforce_name_key_list, email_column = 'Email'):
  name_column = salesforce_name_key_list[0]
  email_row_index = df[name_column].str.contains('@', na=False)
  df['Name_from_email'] = df[email_row_index][email_column].apply(extract_name_from_email)
  return df


def make_composite_key(df_conference, df_all, salesforce_composite_key_list, salesforce_name_key_list):
    attendees_df_llm = df_conference.copy() # attendees data
    customers_df_llm = df_all.copy()  # salesforce data

    # Fill na values with empty strings to make sure the Encoding model works
    attendees_df_llm['First Name'] = attendees_df_llm['First Name'].fillna('').astype(str)
    attendees_df_llm['Last Name'] = attendees_df_llm['Last Name'].fillna('').astype(str)
    attendees_df_llm['Institution'] = attendees_df_llm['Institution'].fillna('').astype(str)

    # if len(salesforce_name_key_list) == 1:
    #   customers_df_llm = identify_and_replace_email_leads(customers_df_llm, salesforce_name_key_list, email_column = 'Email')

    for key in salesforce_composite_key_list:
        customers_df_llm[key] = customers_df_llm[key].fillna('').astype(str)

    # Create Composite Keys
    attendees_df_llm['Composite Key'] = attendees_df_llm['First Name'] + ' ' + attendees_df_llm['Last Name'] + ' ' + attendees_df_llm['Institution']
    customers_df_llm['Composite Key'] = customers_df_llm.apply(lambda row: ' '.join([row[key] for key in salesforce_composite_key_list]), axis=1)
    return (attendees_df_llm, customers_df_llm)


# To process batch data
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]


# Function that compute embeddings for each batch in the data
def compute_embeddings(model, device, text_list, batch_size=32):
    dataset = TextDataset(text_list)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    for batch in dataloader:
        batch_embeddings = model.encode(batch, convert_to_tensor=True, show_progress_bar=False, device=device)
        embeddings.append(batch_embeddings)

    return torch.cat(embeddings)

# Function that pass keys to compute_embeddings() function
def get_embeddings(model, device, attendees_df_llm, customers_df_llm, salesforce_name_key_list, salesforce_institution_key_list, batch_size=32):
    # Get embeddings for the Composite Key, Name, Institution respectively
    attendees_composite_embeddings = compute_embeddings(model, device, attendees_df_llm['Composite Key'].tolist(), batch_size)
    customers_composite_embeddings = compute_embeddings(model, device, customers_df_llm['Composite Key'].tolist(), batch_size)

    attendees_name_embeddings = compute_embeddings(model, device, (attendees_df_llm['First Name'] + ' ' + attendees_df_llm['Last Name']).tolist(), batch_size)
    customers_name_embeddings = compute_embeddings(model, device, customers_df_llm.apply(lambda row: ' '.join([row[key] for key in salesforce_name_key_list]), axis=1).tolist(), batch_size)

    attendees_institution_embeddings = compute_embeddings(model, device, attendees_df_llm['Institution'].tolist(), batch_size)
    customers_account_embeddings = compute_embeddings(model, device, customers_df_llm.apply(lambda row: ' '.join([row[key] for key in salesforce_institution_key_list]), axis=1).tolist(), batch_size)

    return (attendees_composite_embeddings, customers_composite_embeddings, attendees_name_embeddings, customers_name_embeddings, attendees_institution_embeddings, customers_account_embeddings)


# Match
def match_records(attendees_composite_embeddings, customers_composite_embeddings, attendees_name_embeddings, customers_name_embeddings, attendees_institution_embeddings, customers_account_embeddings, attendees_df_llm, customers_df_llm, lower_bound_threshold=0.5, threshold=0.90):
    results = []
    for i, attendee_embedding in enumerate(attendees_composite_embeddings):

        # Calculate the cosine similarity scores for each composite key
        composite_scores = util.pytorch_cos_sim(attendee_embedding, customers_composite_embeddings)
        max_composite_score, max_composite_idx = torch.max(composite_scores, dim=1)

        # Caclulate the cosine similarity scores for the name
        name_scores = util.pytorch_cos_sim(attendees_name_embeddings[i], customers_name_embeddings)
        max_name_score = name_scores[0, max_composite_idx].item()

        # Calculate the cosine similarity scores for the institution
        institution_scores = util.pytorch_cos_sim(attendees_institution_embeddings[i], customers_account_embeddings)
        max_institution_score = institution_scores[0, max_composite_idx].item()

        # If ANY of the scores are below the lower bound threshold, then it is considered an unmatched record
        if max_composite_score.item() < lower_bound_threshold or max_name_score < lower_bound_threshold or max_institution_score < lower_bound_threshold:
            results.append((attendees_df_llm.iloc[i]['Composite Key'], None, max_composite_score.item(), max_name_score, max_institution_score, "Unmatched"))
        # If ALL of the scores are above the threshold, then it is considered a matched record
        elif max_composite_score.item() >= threshold and max_name_score >= threshold and max_institution_score >= threshold:
            best_match = customers_df_llm.iloc[max_composite_idx.item()]['Composite Key']
            results.append((attendees_df_llm.iloc[i]['Composite Key'], best_match, max_composite_score.item(), max_name_score, max_institution_score, "Matched"))
        # If ANY of the scores are below the review threshold, then it is considered a review record
        else:
            best_match = customers_df_llm.iloc[max_composite_idx.item()]['Composite Key']
            results.append((attendees_df_llm.iloc[i]['Composite Key'], best_match, max_composite_score.item(), max_name_score, max_institution_score, "Review"))

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results, columns=['Attendee Composite Key', 'Matched Customer Composite Key', 'Composite Similarity Score', 'Name Similarity Score', 'Institution Similarity Score', 'Status'])
    return results_df


def update_id(row, salesforce_id):
    # if (row['Status'] == 'Matched') | (row['Review'] == 1) : # matched & unmatched (NaN) or reviewed to be matched (1)
    if row['Review'] != 0:
        return row[salesforce_id + ' Customer']
    return

In [33]:
def get_model():
  # Load Model
  model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
  return model


def get_matching_output(model, device, salesforce_client_type_list, salesforce_feature_dict_list, ind):
  salesforce_client_type = salesforce_client_type_list[ind]
  salesforce_feature_dict = salesforce_feature_dict_list[ind]

  print('-'*100)
  print(f'Process starts! Matching E&F data with Salesforce all_{salesforce_client_type} data...')
  print('-'*100)

  # Read data
  salesforce_data_path = f'2024_06_26_salesforce_all_{salesforce_client_type}.csv'
  df_salesforce = read_salesforce(salesforce_data_path)

  if ind == 0:
    enf_data_path = '2024_06_26_e&f_june_conference_attendee_full_list.csv'
  else:
    enf_data_path = f'e&f_unmatched_with_{salesforce_client_type_list[ind-1]}.csv'

  unmatched_enf = pd.read_csv(enf_data_path)

  all_composite_key_list = salesforce_feature_dict.get('salesforce_composite_key_list')
  all_name_key_list = salesforce_feature_dict['salesforce_name_key_list']
  all_institution_key_list = salesforce_feature_dict['salesforce_institution_key_list']
  salesforce_id = salesforce_feature_dict['salesforce_id']

  # Make composite keys
  attendees_df_llm, customers_df_llm = make_composite_key(unmatched_enf, df_salesforce, all_composite_key_list, all_name_key_list)


  # Get embeddings for the composite keys, names, and institutions respectively
  # This should take approx 10-15 mins if gpu else 30+ mins.
  attendees_composite_embeddings, customers_composite_embeddings, attendees_name_embeddings, customers_name_embeddings, attendees_institution_embeddings, customers_account_embeddings = get_embeddings(model, device, attendees_df_llm, customers_df_llm, all_name_key_list, all_institution_key_list)

  # Match records
  results_df = match_records(attendees_composite_embeddings, customers_composite_embeddings, attendees_name_embeddings, customers_name_embeddings, attendees_institution_embeddings, customers_account_embeddings, attendees_df_llm, customers_df_llm, lower_bound_threshold=0.5, threshold=0.90)
  assert len(results_df) == len(attendees_df_llm), "Length of results_df doesn't match length of attendees_df_llm. There may be duplicates in each data frame."
  results_df.drop_duplicates(subset=['Attendee Composite Key'], keep='first', inplace=True)
  attendees_df_llm.drop_duplicates(subset=['Composite Key'], keep='first', inplace=True)
  assert len(results_df) == len(attendees_df_llm), "Lengths still unmatched after dropping duplicates."


  ## Step 1. Merge the results with the attendees data
  output = results_df.merge(attendees_df_llm, how="left", left_on='Attendee Composite Key', right_on='Composite Key')
  # Drop duplicates
  if output.duplicated(subset=['Attendee Composite Key']).sum() > 0:
      output.drop_duplicates(subset=['Attendee Composite Key'], keep='first', inplace=True)
  assert len(results_df) == len(output), "Merging back E&F (attendees) data goes wrong."
  output = output.drop(columns=['Composite Key'])
  ## Delete Salesforce column later for other conference data. Here, Salesforce is just a (assume true) label for accuracy checking
  # output = output.rename(columns={'First Name': 'First Name Attendee', 'Last Name': 'Last Name Attendee', 'Institution': 'Institution Attendee', 'Salesforce': 'Salesforce Attendee'})
  output = output.rename(columns={'First Name': 'First Name Attendee', 'Last Name': 'Last Name Attendee', 'Institution': 'Institution Attendee'})
  len_after_merge_1 = len(output)

  ## Step 2. Merge the results with the customers data
  output = output.merge(customers_df_llm, how="left", left_on='Matched Customer Composite Key', right_on='Composite Key') # Contact ID comes from this table
  # Drop duplicates
  if output.duplicated(subset=['Attendee Composite Key']).sum() > 0:
      output.drop_duplicates(subset=['Attendee Composite Key'], keep='first', inplace=True)
  assert len_after_merge_1 == len(output), "Merging back Salesforce data goes wrong."

  # Drop all the columns in customers_df_llm except for salesforce_id
  output = output.drop(columns=['Composite Key'])
  output = output.drop(columns=all_composite_key_list)
  # output = output.rename(columns={'Contact ID': 'Contact ID Customer'})
  output = output.rename(columns={salesforce_id: salesforce_id + ' Customer'})

  cols_to_keep = results_df.columns.tolist()
  cols_to_keep.extend(['First Name Attendee',	'Last Name Attendee',	'Institution Attendee', salesforce_id + ' Customer'])
  output = output[cols_to_keep]
  output.to_excel(f'output_all_{salesforce_client_type}.xlsx', index=False)


  while True:
    print('')
    print(f'''
    ----------------------------------------------------------------------------
    output_all_{salesforce_client_type}.xlsx is saved.
    ----------------------------------------------------------------------------
    1. Open `output_all_{salesforce_client_type}.xlsx`.
    2. Add a new column called `Review`.
    3. For entries with `Status` being `Review`, fill 1 if reviewed to be matched, 0 if unmatched.
    4. Save the file as `output_all_{salesforce_client_type}_after_review.xlsx`.
    ----------------------------------------------------------------------------
    ''' )

    status = input("Review done? (y/n): ")
    if status.lower() == 'y':
      break
    else:
      continue


  output_after_review = pd.read_excel(f'output_all_{salesforce_client_type}_after_review.xlsx')
  output_after_review[salesforce_id + ' Customer'] = output_after_review.apply(update_id, salesforce_id=salesforce_id, axis=1)
  rest_of_enf = output_after_review[output_after_review[salesforce_id + ' Customer'].isnull()][['First Name Attendee',	'Last Name Attendee',	'Institution Attendee']]
  rest_of_enf.rename(columns={'First Name Attendee': "First Name", 'Last Name Attendee': "Last Name", 'Institution Attendee': "Institution"}, inplace=True)
  rest_of_enf.to_csv(f'e&f_unmatched_with_{salesforce_client_type}.csv', index=False)
  print('----------------------------------------------------------------------------')
  print(f'e&f_unmatched_with_{salesforce_client_type}.csv is saved.')

  ready_for_upload = output_after_review[[salesforce_id + ' Customer']].rename(columns={salesforce_id + ' Customer': salesforce_id})
  # ready_for_upload = ready_for_upload.rename(columns={'First Name Attendee': "First Name", 'Last Name Attendee': "Last Name", 'Institution Attendee': "Institution",'Contact ID Customer': 'Contact ID'})
  ready_for_upload = ready_for_upload[~ready_for_upload[salesforce_id].isnull()]
  ready_for_upload.to_excel(f'ready_for_upload_all_{salesforce_client_type}.xlsx', index=False)

  print(f'''
  ----------------------------------------------------------------------------
  ready_for_upload_all_{salesforce_client_type}.xlsx is saved.
  ----------------------------------------------------------------------------
  Matching finished.
  ''')
  return

In [20]:
enf_data_path = '2024_06_26_e&f_june_conference_attendee_full_list.csv'
salesforce_client_type_list = ['contacts', 'leads', 'unidentified_leads']
salesforce_feature_dict_list = [{'salesforce_client_type': 'contacts',
  'salesforce_composite_key_list': ['First Name', 'Last Name', 'Account Name'],
  'salesforce_name_key_list': ['First Name', 'Last Name'],
  'salesforce_institution_key_list': ['Account Name'],
  'salesforce_id': 'Contact ID'
  },

 {'salesforce_client_type': 'leads',
  'salesforce_composite_key_list': ['First Name', 'Last Name', 'Account'],
  'salesforce_name_key_list': ['First Name', 'Last Name'],
  'salesforce_institution_key_list': ['Account'],
  'salesforce_id': 'Lead ID'},

 {'salesforce_client_type': 'unidentified_leads',
  'salesforce_composite_key_list': ['Unidentified Lead: Unidentified Leads', 'Firm Name'],
  'salesforce_name_key_list': ['Unidentified Lead: Unidentified Leads'],
  'salesforce_institution_key_list': ['Firm Name'],
  'salesforce_id': 'Unidentified Lead: ID'}
]

In [34]:

# Check availability for gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('-'*100)
print("Device using is", device)
print('-'*100)

# Get pre-trained model
model = get_model()
model.to(device)

get_matching_output(model, device, salesforce_client_type_list, salesforce_feature_dict_list, ind = 2)

----------------------------------------------------------------------------------------------------
Device using is cuda
----------------------------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


----------------------------------------------------------------------------------------------------
Process starts! Matching E&F data with Salesforce all_unidentified_leads data...
----------------------------------------------------------------------------------------------------
Successfully read the CSV file with latin1 encoding.


    ----------------------------------------------------------------------------
    output_all_unidentified_leads.xlsx is saved. 
    ----------------------------------------------------------------------------
    1. Open `output_all_unidentified_leads.xlsx`.
    2. Add a new column called `Review`.
    3. For entries with `Status` being `Review`, fill 1 if reviewed to be matched, 0 if unmatched.
    4. Save the file as `output_all_unidentified_leads_after_review.xlsx`.
    ----------------------------------------------------------------------------
    
Review done? (y/n): y
----------------------------------------------------------------------------
